In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00


In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.4 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
from datasets import load_dataset

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [ ]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [ ]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(f'Categories: {classes_list}')

Categories: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
print(len(train_data[0]))
print(len(train_data[1]))

701
701


In [ ]:
print(len(val_data[0]))
print(len(val_data[1]))

99
99


In [ ]:
print(len(test_data[0]))
print(len(test_data[1]))

204
204


In [ ]:
nlp = spacy.load('ru_core_news_sm')

In [ ]:
print(train_data[0][0])

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным.


In [ ]:
print(normalize_text(train_data[0][0], nlp))

турция с <NUM> сторона окружить море на запад эгейский на север чёрный и на юг средиземный


In [ ]:
print(val_data[0][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [ ]:
print(normalize_text(val_data[0][0], nlp))

если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость


In [ ]:
print(test_data[0][0])

Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.


In [ ]:
print(normalize_text(test_data[0][0], nlp))

мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [ ]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability
print(f'Maximal document frequency of term is {max_df}.')

Maximal document frequency of term is 0.9714285714285714.


In [ ]:
# classifier = Pipeline(steps=[
#     ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1)),
#     ('cls', LogisticRegression(solver='saga', max_iter=100, random_state=42))
# ])

In [ ]:
# cv = GridSearchCV(
#     estimator=classifier,
#     param_grid={
#         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
#         'cls__C': [1e-1, 1, 10, 100, 1000],
#         'cls__penalty': ['l1', 'l2']
#     },
#     scoring='f1_macro',
#     cv=5,
#     refit=True,
#     n_jobs=-1,
#     verbose=True
# )

In [ ]:
# cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

In [ ]:
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Загружаем данные
train_data, val_data, test_data, classes_list = load_sib200_ru()

# Создаем модель CatBoost в Pipeline
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(token_pattern=r'\w+', max_df=max_df, min_df=1)),
    ('cls', CatBoostClassifier(verbose=0, random_seed=42))
])

# Параметры для поиска
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    # 'cls__iterations': [500, 1000],
    'cls__iterations': [100],                      # Количество итераций обучения
    'cls__learning_rate': [0.01, 0.05, 0.1],
    'cls__depth': [6, 8, 10]
}

# Настраиваем GridSearchCV
cv = GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1,
    verbose=True
)

# Обучаем модель
cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

# Предсказания на валидационном наборе
y_pred_val = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print("Validation Report:")
print(classification_report(y_true=val_data[1], y_pred=y_pred_val, target_names=classes_list))

# Предсказания на тестовом наборе
y_pred_test = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print("Test Report:")
print(classification_report(y_true=test_data[1], y_pred=y_pred_test, target_names=classes_list))


Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [ ]:
print('Best parameters:')
print(cv.best_params_)

In [ ]:
print('Best F1-macro:')
print(cv.best_score_)

In [ ]:
print(f'Vocabulary size is {len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_)}.')

In [ ]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

In [ ]:
y_pred = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

оптимизаторы

регулязаторы

ПОСМОТРЕТЬ: библиотека optuna

методы мл cat boost, back of words, ngrama


Оптимизаторы — это методы, которые используются для минимизации или максимизации целевой функции в процессе обучения моделей машинного обучения. Они играют ключевую роль в процессе настройки параметров модели для достижения наилучших результатов.

* Gradient Descent (Градиентный спуск): Один из наиболее распространенных оптимизаторов, который обновляет веса модели путем итеративного вычисления градиента целевой функции по отношению к параметрам и последующего смещения параметров в направлении, уменьшающем значение этой функции. Варианты:
    * Stochastic Gradient Descent (SGD): Применяет градиентный спуск на каждом примере данных, что может ускорить обучение на больших наборах данных, но делает процесс более шумным.
    * Mini-batch Gradient Descent: Компромисс между обычным градиентным спуском и стохастическим, обновляет веса на основе небольших батчей данных.
    * Batch Gradient Descent: Использует весь набор данных для одного обновления весов, что может быть медленно на больших объемах данных.

* Adam (Adaptive Moment Estimation): Современный оптимизатор, который адаптирует шаг обучения для каждого параметра, используя первые и вторые моменты градиента. Он обычно работает лучше и быстрее на практике, чем классический градиентный спуск.
    * Комбинирует лучшие качества AdaGrad и RMSProp.
    * Использует две скорости обучения: для градиентов и для квадратов градиентов, что позволяет более эффективное обновление весов.

* RMSprop: Оптимизатор, который разделяет шаг обучения на скользящее среднее квадрата градиентов, тем самым предотвращая колебания, характерные для стандартного градиентного спуска.

* SAGA и Newton-CG: Эти оптимизаторы часто используются в логистической регрессии и линейных моделях.
    * SAGA — вариант стохастического оптимизатора, особенно эффективен для задач с большими и разреженными матрицами признаков.
    * Newton-CG — использует метод Ньютона для быстрого нахождения минимума, особенно эффективен для выпуклых задач.

# Регуляризация

Регуляризация — это методы, используемые для предотвращения переобучения (overfitting) в моделях машинного обучения. Они добавляют штрафы к целевой функции, чтобы сдерживать чрезмерно сложные модели, которые могут плохо обобщать данные.


* L1-регуляризация (Lasso): Добавляет к функции потерь сумму абсолютных значений коэффициентов модели. Это приводит к тому, что некоторые коэффициенты становятся равными нулю, что делает L1 полезным для отбора признаков.
Пример: В логистической регрессии или линейной регрессии L1-регуляризация может исключать нерелевантные признаки, улучшая обобщающую способность модели.

* L2-регуляризация (Ridge): Добавляет к функции потерь сумму квадратов коэффициентов модели. В отличие от L1, L2 не зануляет коэффициенты, а лишь делает их меньше, сглаживая влияние отдельных признаков.
Пример: Используется в методах, таких как логистическая регрессия и SVM, для уменьшения переобучения.

* Elastic Net: Комбинация L1 и L2 регуляризации. Применяется, когда модель должна учитывать как зануление нерелевантных признаков (L1), так и сглаживание больших коэффициентов (L2).

In [ ]:
# # !pip install datasets optuna
# # !python -m spacy download ru_core_news_sm
#
# from typing import List, Tuple
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report
# from sklearn.pipeline import Pipeline
# import spacy
# from datasets import load_dataset
# import optuna
# from sklearn.model_selection import cross_val_score
#
# def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
#     trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
#     X_train = trainset['text']
#     y_train = trainset['category']
#     valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
#     X_val = valset['text']
#     y_val = valset['category']
#     testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
#     X_test = testset['text']
#     y_test = testset['category']
#     categories = set(y_train)
#     unknown_categories = set(y_val) - categories
#     if len(unknown_categories) > 0:
#         err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
#         raise RuntimeError(err_msg)
#     unknown_categories = set(y_test) - categories
#     if len(unknown_categories) > 0:
#         err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
#         raise RuntimeError(err_msg)
#     categories = sorted(list(categories))
#     y_train = [categories.index(it) for it in y_train]
#     y_val = [categories.index(it) for it in y_val]
#     y_test = [categories.index(it) for it in y_test]
#     return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories
#
# def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
#     doc = nlp_pipeline(s)
#     lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
#     if len(lemmas) == 0:
#         return ''
#     return ' '.join(lemmas)
#
# train_data, val_data, test_data, classes_list = load_sib200_ru()
# nlp = spacy.load('ru_core_news_sm')
# class_probability = 1.0 / len(classes_list)
# max_df = 1.0 - 0.2 * class_probability
#
# def objective(trial):
#     ngram_range = trial.suggest_categorical("ngram_range", [(1, 1), (1, 2), (1, 3)])
#     C = trial.suggest_loguniform("C", 1e-4, 1e3)
#     penalty = trial.suggest_categorical("penalty", ['l2'])
#
#     classifier = Pipeline(steps=[
#         ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1, ngram_range=ngram_range)),
#         ('cls', LogisticRegression(solver='newton-cholesky', max_iter=100, random_state=42, C=C, penalty=penalty))
#     ])
#
#     score = cross_val_score(classifier, [normalize_text(it, nlp) for it in train_data[0]], train_data[1], cv=5, scoring='f1_macro', n_jobs=-1).mean()
#
#     return score
#
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=3)
#
# best_params = study.best_trial.params
# print(f"Best trial: {study.best_trial.value}")
# print(f"Best parameters: {study.best_trial.params}")
#
# final_classifier = Pipeline(steps=[
#     ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1, ngram_range=best_params['ngram_range'])),
#     ('cls', LogisticRegression(solver='newton-cholesky', max_iter=100, random_state=42, C=best_params['C'], penalty=best_params['penalty']))
# ])
#
# final_classifier.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])
#
# y_pred_val = final_classifier.predict([normalize_text(it, nlp) for it in val_data[0]])
# print(classification_report(y_true=val_data[1], y_pred=y_pred_val, target_names=classes_list))
#
# y_pred_test = final_classifier.predict([normalize_text(it, nlp) for it in test_data[0]])
# print(classification_report(y_true=test_data[1], y_pred=y_pred_test, target_names=classes_list))
